import data and seperate x, y, train validation

In [2]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_validation = pd.read_csv('validation.csv')
df_test = pd.read_csv('test.csv')

# print(df_train.head())

X_train = df_train.iloc[:,range(0,35)]
y_train = df_train.iloc[:,35]
#print(X_train.head())
# print(y_train.head())

X_validation = df_validation.iloc[:,range(0,35)]
y_validation = df_validation.iloc[:,35]
print(X_validation.head())

     Id  Marital status  Application mode  Application order  Course  \
0  1255               4                12                  1       8   
1  3458               1                 8                  1      10   
2  3390               1                 8                  1      16   
3  1497               1                 8                  2      14   
4  1536               1                12                  1      12   

   Daytime/evening attendance  Previous qualification  Nacionality  \
0                           1                       1            1   
1                           1                       1            1   
2                           1                       1            1   
3                           1                       1            1   
4                           1                       1            1   

   Mother's qualification  Father's qualification  ...  \
0                       3                       1  ...   
1                       4     

ensemble

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import VotingClassifier

SEED = 1

lr = LogisticRegression()
knn = KNN()
dt = DecisionTreeClassifier()

classifiers = [('Logistic Regression', lr),('K Nearest Neighbours', knn),('Classification Tree', dt)]

for clf_name, clf in classifiers:
    clf.fit(X_train, y_train)

    y_pred =clf.predict(X_validation)
    print(accuracy_score(y_validation,y_pred))

vc = VotingClassifier(estimators=classifiers)

vc.fit(X_train, y_train)
y_pred = vc.predict(X_validation)
print(accuracy_score(y_validation,y_pred))





/home/jack/students_dropout/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6995481927710844
0.5579819277108434
0.6814759036144579
0.7115963855421686


/home/jack/students_dropout/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


random forests

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=500, min_samples_leaf=0.12)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_validation)
print(accuracy_score(y_validation,y_pred))

0.6957831325301205


polynominal regression

In [10]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import math



degree = 2

polyreg=make_pipeline(
    PolynomialFeatures(degree),
    LinearRegression(),
)

polyreg.fit(X_train, y_train)

y_pred = rf.predict(X_validation)
print(accuracy_score(y_validation,y_pred))

ValueError: could not convert string to float: 'Graduate'

boosting

In [17]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt = DecisionTreeClassifier(max_depth=6)
adb_clf = AdaBoostClassifier(base_estimator=dt, n_estimators=300)

adb_clf.fit(X_train, y_train)
y_pred = adb_clf.predict(X_validation)
print(accuracy_score(y_validation,y_pred))

0.7364457831325302


Stochastic GradientBoosting

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

sgbr = GradientBoostingClassifier(max_depth=4,subsample=0.9,max_features=0.6,n_estimators=200)

sgbr.fit(X_train, y_train)
y_pred = sgbr.predict(X_validation)
print(accuracy_score(y_validation,y_pred))

0.7673192771084337


tune hyperparameters

In [21]:
from sklearn.model_selection import GridSearchCV

params_sgbr = {

    'max_depth': [3,4,5,6],
    'subsample': [0.7,0.8,0.9],
    'max_features': [0.4,0.6,0.8],
    'n_estimators': [200,300,400]
}

grid_sgbr = GridSearchCV(estimator=sgbr,param_grid=params_sgbr,scoring='accuracy',cv=10,n_jobs=-1)
grid_sgbr.fit(X_train, y_train)
print(grid_sgbr.best_params_)

{'max_depth': 4, 'max_features': 0.6, 'n_estimators': 200, 'subsample': 0.9}
